In [ ]:
# Set our STAC API URL
STAC=https://earth-search.aws.element84.com/v1

# Disable AWS client authentication
export AWS_NO_SIGN_REQUEST=YES

In [ ]:
# See what collections we have in the catalog
stac-client collections $STAC | jq '.[].id'

In [ ]:
# Get the metadata for the Copernicus 30m DEM
stac-client collections $STAC | jq '.[] | select(.id == "cop-dem-glo-30")'

In [ ]:
# Find the DEM tile that intersects Mt. Hood AOI
ITEM="$(stac-client search $STAC --intersects ../aois/mthood.geojson --collection cop-dem-glo-30)"
<<<$ITEM jq .

In [ ]:
# Extract the geotransform values from the item's projection metadata
TRANSFORM="$(<<<$ITEM jq '.features[].properties."proj:transform"[]')"
eval $(
    <<<$TRANSFORM awk '
        {print "PX_WIDTH="tolower($1)};
        {print "ROW_ROT="tolower($2)};
        {print "UP_LEFT_LONG="tolower($3)};
        {print "COL_ROT="tolower($4)};
        {print "PX_HEIGHT="tolower($5)};
        {print "UP_LEFT_LAT="tolower($6)};
    '
)

echo $UP_LEFT_LONG, $UP_LEFT_LAT

In [ ]:
# The summit of Mt. Hood is at -121.695833, 45.373611 (https://en.wikipedia.org/wiki/Mount_Hood).
SUMMIT_LONG=-121.695833
SUMMIT_LAT=45.373611

# Calculate the pixel coords of the summit based on the item's geotransform
# (we use python for the arithmetic because bash doesn't support floats)
# (the cut command effectively floors the result by truncating to an int)
SUMMIT_COL=$(python -c "print(($SUMMIT_LONG - $UP_LEFT_LONG) / $PX_WIDTH)" | cut -d '.' -f 1)
SUMMIT_ROW=$(python -c "print(($SUMMIT_LAT - $UP_LEFT_LAT) / $PX_HEIGHT)" | cut -d '.' -f 1)

echo $SUMMIT_COL, $SUMMIT_ROW

In [ ]:
# Extract the href for the item's data asset, replacing the scheme for use with GDAL
HREF="$(<<<$ITEM jq -r '.features[].assets.data.href' | sed 's|s3://|/vsis3/|')"

echo $HREF

In [ ]:
# We can use GDAL to get the value for the summit cell directly from the remote asset
time gdallocationinfo "$HREF" $SUMMIT_COL $SUMMIT_ROW

In [ ]:
time gdalinfo $HREF

In [ ]:
time gdalinfo -stats $HREF

In [ ]:
time gdal_translate -outsize 10% 10% --debug on $HREF out.tif

In [ ]:
gdalinfo out.tif